In [1]:
import wfdb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import fftpack, stats, signal
from scipy.optimize import curve_fit
import scipy.linalg as la
from scipy import signal
import pywt
from scipy import stats

In [2]:
def segmentation(ann):
# function that performs segmentation based on the relative known proportions between RR, PR and QT 
# the function returns
    
    # I initialize the variables I need for the while loop
    patient_start = np.ones(2)*-3
    i = 1
    j = -1
    while patient_start[0]<0:
        j += 1
        # I calculate the RR intervals
        RR_intervals = (ann[(j+i):]-ann[j:-i])
        # firstly, I crop the RR interval to match the relative proportion of RR:QT=0.8:0.41 
        QT_size = (RR_intervals*0.41/0.8).astype(int)
        PR_size = (RR_intervals*0.16/0.8).astype(int)
        patient_start = ann[j:-i] - PR_size
        patient_end = ann[j:-i] + QT_size
    # j is the number of annotations ignored from the beginning
    # i is the number of annotations ignored from the end
    # so that, for compatibility purposes, I will refer to
    return QT_size, PR_size, patient_start, patient_end, i, j

In [3]:
r = list(map(int,(np.loadtxt("input_files/RECORDS"))))
records = list(map(str,r))
FILE_SAVE=True

if FILE_SAVE:
    import time
    t1 = time.time()
    
    # we want to save a dataframe of heartbeats: to do that, I initialize some temporary means of storage
    # - the number that identifies the patient
    number = np.array([0])
    # - the number that identifies the heartbeat of each patient
    idx = np.array([0])
    # - the index of the start of the heartbeat
    start = np.array([0])
    # - the index of the end of the heartbeat
    end = np.array([0]) 
    # - the index of the R peak of the heartbeat
    R_peak = np.array([0]) 
    # - the symbol for the annotation of the heartbeat
    annot = [None] 
    
    # I then fill such temporary means of storage by updating them after analyzing every file/patient
    for path in records:
        # UPLOAD
        # data = pd.read_csv("output_files/data"+path+".csv")
        annotations = pd.read_csv("output_files/data"+path+"_ann.csv")
        ann = np.array(annotations['index'])
        
        # and use my custom function for SEGMENTATION
        QT_size, PR_size, patient_start, patient_end, i, j = segmentation(ann)
        # I add the remaining pieces of information
        # - the number of the patient is the same for all its heartbeats
        patient_number = np.ones_like(patient_start)*int(path)
        # - the number that identifies the heartbeat of each patient is a simple increasing vector of integers
        patient_idx = np.arange(0,len(patient_start),1,dtype = int)
        # - the location of the R peaks (by cropping the annotations accordingly)
        patient_R_peak = ann[j:-i]
        # - the symbols for the annotations (by cropping the annotations accordingly)
        patient_ann = list(annotations['symbol'][j:-i])
        
        # lastly, I check that all the annotations are in between the corresponding starts and stops
        check = np.all([(patient_R_peak>patient_start),(patient_R_peak<patient_end)])
        if not check:
            print(check)
           
        # and update the temporary means of storage
        number = np.concatenate((number, patient_number))
        idx = np.concatenate((idx, patient_idx))
        start = np.concatenate((start, patient_start))
        end = np.concatenate((end, patient_end))
        R_peak = np.concatenate((R_peak, patient_R_peak))
        annot.extend(patient_ann)
    
    # finally, I put together the heartbeat dataframe  
    hb = pd.DataFrame({"patient": number[1:], "label": idx[1:], "start": start[1:], "end": end[1:],
                       "R_peak": R_peak[1:], "ann": annot[1:] })
    t2 = time.time()
    print("the algorithm took",t2-t1)
    outfn = 'output_files/heartbeats.csv'
    hb.to_csv(outfn, index=False)
else:
    hb = pd.read_csv('output_files/heartbeats.csv')
    
    display(hb)

the algorithm took 0.14143991470336914
